# svg_dataset

> End-to-end functions taking in centerline-stroke SVG's and outputting deltas in Stroke-3 format.

In [ ]:
#| default_exp stroke3.svg_dataset

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
# |hide

In [ ]:
#| export


from singleline_dataset.stroke3.path_transforms import svg_to_strokes
from singleline_dataset.stroke3.stroke3 import strokes_to_deltas, rdp_strokes


def stroke_rdp_deltas(rescaled_strokes, epsilon=2.0):
    rdp_strokes = rdp_strokes(rescaled_strokes, epsilon)
    deltas = strokes_to_deltas(rdp_strokes)

    ## roundtrip / sanity check
    # _rdp_strokes = stroke3.deltas_to_strokes(deltas)
    # display_plot.plot_strokes(_rdp_strokes)

    return deltas, rescaled_strokes

In [ ]:
#| export
import os

import numpy as np

from singleline_dataset.stroke3.display_svg import render_deltas, render_strokes
from singleline_dataset.stroke3.svg_files import enumerate_files


def svgs_to_deltas(input_dir, output_dir=None, target_size=200, total_n=1000, min_n=3, epsilon=1.0):
    if output_dir and not os.path.isdir(output_dir):
        os.makedirs(output_dir)

    dataset = []
    for fname in enumerate_files(input_dir):
        input_fname = os.path.join(input_dir, fname)

        try:
            rescaled_strokes = svg_to_strokes(input_fname, total_n=total_n, min_n=min_n)
            deltas = stroke_rdp_deltas(rescaled_strokes, epsilon=epsilon)
            dataset.append(deltas)

            # monitor number of points before/after applying RDP path simplification algorithm
            raw_points = np.vstack(rescaled_strokes).shape[0]
            rdp_points = deltas.shape[0]
            print(f"{input_fname} points: raw={raw_points}, rdp={rdp_points}")

            if output_dir:
                raw_output_fname = os.path.join(
                    output_dir, fname.replace(".svg", ".raw.svg")
                )
                preproc_output_fname = os.path.join(
                    output_dir, fname.replace(".svg", ".preproc.svg")
                )

                with open(raw_output_fname, "w", encoding="utf-8") as raw_out:
                    raw_dwg = render_strokes(rescaled_strokes, target_size=target_size)
                    raw_dwg.write(raw_out, pretty=True)
                    print(f"\twrote {raw_output_fname}")

                with open(preproc_output_fname, "w", encoding="utf-8") as preproc_out:
                    preproc_dwg = render_deltas(deltas, target_size=target_size)
                    preproc_dwg.save(preproc_output_fname)
                    print(f"\twrote {preproc_output_fname}")
        except Exception as e:
            print(f"error processing {input_fname}: {e}")
    return np.array(dataset, dtype=object)

In [ ]:
#| hide
import nbdev

nbdev.nbdev_export()